# Imports

In [ ]:
import os
from pathlib import Path

import numpy as np
import pandas as pd
import pypsa
from pandas.testing import assert_frame_equal

In [ ]:
if "notebooks" in os.getcwd():
    %cd ..
scenario = "NT"
pyear = 2040
cyear = 2013

fn_n = Path(f"resources/tyndp/networks/base_s_all___{pyear}_brownfield.nc")
if scenario != "NT":
    fn_raw = Path(
        f"data/tyndp_2024_bundle/Demand Profiles/{scenario}/{pyear}/ELECTRICITY_MARKET {scenario} {pyear}.xlsx"
    )
else:
    fn_raw = Path(
        f"data/tyndp_2024_bundle/Demand Profiles/{scenario}/Electricity demand profiles/{pyear}_National Trends.xlsx"
    )

# Read network

In [ ]:
n = pypsa.Network(fn_n)
buses = n.buses.query("carrier=='AC' and not Bus.str.contains('DRES')").index

In [ ]:
n.loads_t.p_set.head()

# Read raw file

In [ ]:
cyear = cyear if scenario == "NT" else 2009
data = pd.read_excel(
    fn_raw,
    skiprows=11 if scenario != "NT" else 7,
    index_col=0,
    usecols=lambda name: name == "Date" or name == cyear,
    sheet_name=None,
)

In [ ]:
if scenario in ["DE", "GA"]:
    data["PL00"].index = data["AL00"].index
    cyear_ = cyear if pyear == 2030 else cyear - 1
    data["UK00"] = pd.read_excel(
        fn_raw,
        skiprows=11 if scenario != "NT" else 7,
        index_col=0,
        usecols=lambda name: name == "Date" or name == cyear_,
        sheet_name="UK00",
    )

In [ ]:
demand = pd.concat(data, axis=1).droplevel(1, axis=1)
demand.index = demand.index.map(lambda t: t.replace(year=2013))
demand = demand.loc[f"{cyear}-03-01" : f"{cyear}-03-07"]

In [ ]:
aggregation_map = (
    pd.Series(n.snapshot_weightings.index.get_indexer(demand.index), index=demand.index)
    .replace(-1, np.nan)
    .ffill()
    .astype(int)
    .map(lambda i: n.snapshot_weightings.index[i])
)
demand_agg = demand.groupby(aggregation_map).mean()
demand_agg.columns = demand.columns.str.replace("UK", "GB")
demand_agg = demand_agg[[c for c in demand_agg.columns if c in buses]]
demand_agg.index.name = "snapshot"
demand_agg.columns.name = "Load"

demand_agg.head()

# Compare

In [ ]:
assert_frame_equal(n.loads_t.p_set, demand_agg, check_like=True)